# Data Extract and upload for NADAbase

## Steps

1. Extract Episodes for the period from Communicare
2. Extract ATOM data
3. Match Assessments with Episodes
4. Generate the upload Survey.txt file.

In [1]:
# import os

import pandas as pd

# import numpy as np
import mylogger
# from nada_load import load_and_parse_csv
from data_config import EstablishmentID_Program
from utils.df_xtrct_prep import  load_and_parse_csv
from utils.fromstr import convert_format_datestr

logger = mylogger.get(__name__)
# # List of column names in the CSV
# column_names = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID', 'DOB', 'DOB STATUS', 'SEX', 'COUNTRY OF BIRTH', 'INDIGENOUS STATUS', 'PREFERRED LANGUAGE', 'SOURCE OF INCOME', 'LIVING ARRANGEMENT', 'USUAL ACCOMMODATION', 'CLIENT TYPE', 'PRINCIPAL DRUG OF CONCERN', 'PDCSubstanceOfConcern', 'ILLICIT USE', 'METHOD OF USE PRINCIPAL DRUG', 'INJECTING DRUG USE', 'SETTING', 'CommencementDate', 'POSTCODE', 'SOURCE OF REFERRAL', 'MAIN SERVICE', 'EndDate', 'END REASON', 'REFERRAL TO ANOTHER SERVICE', 'FAMILY NAME', 'GIVEN NAME', 'MIDDLE NAME', 'TITLE', 'SLK', 'MEDICARE NUMBER', 'PROPERTY NAME', 'UNIT FLAT NUMBER', 'STREET NUMBER', 'STREET NAME', 'SUBURB']




## Episode Data

In [2]:
# Global variables
active_clients_start_date ='2023-07-01' 
active_clients_end_date = '2023-12-31'

start_dt = convert_format_datestr(active_clients_start_date,'%Y-%m-%d', '%d%m%Y')
end_dt = convert_format_datestr(active_clients_end_date,'%Y-%m-%d', '%d%m%Y')


ep_source_filename = f'DATS_AllPrograms_{start_dt}-{end_dt}' #'DATS_Annual_AllPrograms_FY22-23'
ep_datasource_file_path = f"./data/in/NSW_CSV/{ep_source_filename}.csv"

# extract_start_date = 20130501
# extract_end_date = 20231231

# fname = f"{extract_start_date}_{extract_end_date}" # TODO :forNaada



results_folder = "./data/out/"

In [3]:
# # List of columns we care about
columns_of_interest = ['ESTABLISHMENT IDENTIFIER', 'GEOGRAPHICAL LOCATION', 'EPISODE ID','PERSON ID', 'SPECIFY DRUG OF CONCERN', 'PRINCIPAL DRUG OF CONCERN', 'START DATE', 'END DATE', 'SLK']
rename_columns = {
    'SPECIFY DRUG OF CONCERN': 'PDCSubstanceOfConcern',
    'PRINCIPAL DRUG OF CONCERN': 'PDCCode',
    'START DATE': 'CommencementDate',
    'END DATE': 'EndDate',
    'EPISODE ID': 'PMSEpisodeID',
    'PERSON ID': 'PMSPersonID',    
}

In [4]:
# ep_df  = load_and_parse_episode_csvs(ep_datasource_file_path)
ep_df = load_and_parse_csv(ep_datasource_file_path, rename_columns, columns_of_interest, date_cols=['START DATE', 'END DATE'])
ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)

In [ ]:
# ep_df.head(10)

In [8]:
len(ep_df), min(ep_df.CommencementDate), max(ep_df.CommencementDate)

(797, datetime.date(2020, 4, 7), datetime.date(2023, 12, 22))

In [5]:
# ep_df['Program'] = ep_df['ESTABLISHMENT IDENTIFIER'].map(EstablishmentID_Program)
ep_df.rename(columns={'ESTABLISHMENT IDENTIFIER': 'AgencyCode'}, inplace=True)

In [10]:
ep_df.columns

Index(['AgencyCode', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID',
       'PDCCode', 'PDCSubstanceOfConcern', 'CommencementDate', 'EndDate',
       'SLK', 'Program'],
      dtype='object')

In [ ]:
# ep_df.head()

## ATOM Data


In [6]:
from utils.environment import MyEnvironmentConfig
from utils.df_xtrct_prep import extract_prep_atom_data


In [ ]:
# to adjust the ODC parser
extract_start_date = 20200701
extract_end_date = 20231231
active_clients_start_date ='2022-07-01' 
active_clients_end_date = '2023-12-31'
fname = f"{extract_start_date}-{extract_end_date}" # TODO :forNaada
MyEnvironmentConfig().setup('prod')

processed_df = extract_prep_atom_data(extract_start_date, extract_end_date
                                 , active_clients_start_date
                                 , active_clients_end_date
                                 , fname, purpose='NADA')

In [13]:
# processed_df = ['SLK',	'RowKey','ClientType',	'AssessmentDate', PDCSubstanceOrGambling	PDCMethodOfUse	PDCDaysInLast28	PDCUnits	PDCHowMuchPerOccasion	PDCAgeLastUsed	PDCGoals	PDC]
processed_df.columns

Index(['SLK', 'RowKey', 'Program', 'Staff', 'AssessmentDate',
       'SDSIsAODUseOutOfControl', 'SDSHowMuchDoYouWorryAboutAODUse',
       'SDSDoesMissingFixMakeAnxious', 'SDSDoYouWishToStop',
       'SDSHowDifficultToStopOrGoWithout', 'K10Q01', 'K10Q02', 'K10Q03',
       'K10Q04', 'K10Q05', 'K10Q06', 'K10Q07', 'K10Q08', 'K10Q09', 'K10Q10',
       'AssessmentType', 'Past4WkPhysicalHealth', 'Past4WkMentalHealth',
       'Past4WkHaveYouViolenceAbusive', 'Past4WkBeenArrested',
       'Past4WkQualityOfLifeScore', 'Past4WkNumInjectingDays',
       'Alcohol_DaysInLast28', 'Alcohol_PerOccassionUse',
       'Alcohol_TypicalQtyStr', 'Amphetamines_DaysInLast28',
       'Amphetamines_TypicalQtyStr', 'Another Drug1',
       'Another Drug1_DaysInLast28', 'Another Drug1_TypicalQtyStr',
       'Cannabis_DaysInLast28', 'Cannabis_TypicalQtyStr',
       'Nicotine_DaysInLast28', 'Nicotine_PerOccassionUse',
       'Nicotine_TypicalQtyStr', 'Other Opioids_DaysInLast28',
       'Other Opioids_TypicalQtyStr',

In [14]:
processed_df[processed_df['SLK'] =='EEAOH120519841']

,SLK,RowKey,Program,Staff,AssessmentDate,SDSIsAODUseOutOfControl,SDSHowMuchDoYouWorryAboutAODUse,SDSDoesMissingFixMakeAnxious,SDSDoYouWishToStop,SDSHowDifficultToStopOrGoWithout,...,Another Drug2,Another Drug2_DaysInLast28,Another Drug2_TypicalQtyStr,ATOPRiskEviction,ATOPHomeless,Past4Wk_ViolentToYou,PrimaryCaregiver_0-5,PrimaryCaregiver_5-15,PaidWorkDays,StudyDays


In [15]:
processed_df['Program'].unique()

array(['EUROPATH', 'MURMICE', 'GOLBICE', 'BEGAPATH', 'MONPATH',
       'GOLBGNRL', 'MURMHEAD', 'MURMPP', 'SAPPHIRE', 'MURMWIO'],
      dtype=object)

In [10]:
out = processed_df


In [12]:
out[ [c for c in out.columns if 'Cannabis' in c]] 

,Cannabis_TypicalQtyStr,Cannabis_DaysInLast28
448,NaN,NaN
593,NaN,NaN
580,NaN,NaN
548,NaN,NaN
472,0,0
...,...,...
1104,NaN,NaN
817,3.0; cones / joints,0
629,NaN,NaN
1164,0.5; grams,4


In [7]:
processed_df.K10Q12.value_counts()

K10Q12
18    8
25    3
21    2
1     2
5     2
10    2
15    2
2     2
8     2
13    1
28    1
14    1
3     1
20    1
23    1
Name: count, dtype: int64

In [16]:
# from utils.io import write_parquet
# processed_filepath = f"./data/processed/atom_NADA_{fname}.parquet"
# write_parquet(processed_df, processed_filepath) # don't force overwrite
processed_df.to_csv(f"./data/processed/atom_NADA_{fname}.csv")
# # ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column K10Q12 with type object')

In [62]:
atom_df  = processed_df.copy()
atom_df

,SLK,RowKey,Program,Staff,AssessmentDate,SDSIsAODUseOutOfControl,SDSHowMuchDoYouWorryAboutAODUse,SDSDoesMissingFixMakeAnxious,SDSDoYouWishToStop,SDSHowDifficultToStopOrGoWithout,...,Another Drug2,Another Drug2_DaysInLast28,Another Drug2_TypicalQtyStr,ATOPRiskEviction,ATOPHomeless,Past4Wk_ViolentToYou,PrimaryCaregiver_0-5,PrimaryCaregiver_5-15,PaidWorkDays,StudyDays
2755,MIHEN080119732,EUROPATH_ITSP_01072020,EUROPATH,None,2020-07-01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,0,1,0,0
4210,SACAR270619511,MURMICE_INAS_20200701,MURMICE,Charles.Debaecker,2020-07-01,0.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,None,None,None,None,None,None,None
3406,ORHHA031019932,GOLBICE_ITSP_20200702,GOLBICE,Carol.Sharp,2020-07-02,2.0,2.0,2.0,3.0,1.0,...,NaN,NaN,NaN,None,None,None,None,None,None,None
3369,ORELA041019632,EUROPATH_INAS_10072020,EUROPATH,Vic.May,2020-07-10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,25,0
4600,URUEL110919772,EUROPATH_INAS_13072020,EUROPATH,Vic.May,2020-07-13,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,None,None,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934,ODEMI271219932,SAPPHIRE_INAS_20231222,SAPPHIRE,Kylie.Wood,2023-12-22,2.0,3.0,3.0,3.0,2.0,...,NaN,NaN,NaN,0,0,0,0,0,0,None
69,ADIRO150619781,GOLBICE_ITSP_20231222,GOLBICE,James.Costello,2023-12-22,2.0,2.0,2.0,3.0,1.0,...,NaN,NaN,NaN,0,0,0,0,1,4,0
3961,RENAS110920041,GOLBICE_INAS_20231222,GOLBICE,Anthony.Williams,2023-12-22,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,None,None,None,None,None,None,None
1984,HIPAR140319861,GOLBICE_INAS_20231222,GOLBICE,Anthony.Williams,2023-12-22,2.0,2.0,2.0,3.0,1.0,...,NaN,NaN,NaN,None,None,None,None,None,20,None


In [14]:
# atom_df = atom_df.loc[ (atom_df.AssessmentDate>= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)]

In [18]:
atom_df['AssessmentDate'].info()  #describe()

<class 'pandas.core.series.Series'>
Index: 3147 entries, 2755 to 2880
Series name: AssessmentDate
Non-Null Count  Dtype         
--------------  -----         
3147 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 49.2 KB


In [10]:
# import pandas as pd
# # Ensure that AssessmentDate is in datetime format
# atom_df['AssessmentDate'] = pd.to_datetime(atom_df['AssessmentDate'], errors='coerce')

In [9]:
# atom_df.drop(atom_df[atom_df.Program.isin(['TSS', 'ARCA', 'COCO', 'PSYNSW' ])].index, inplace=True)
# non_nsw_programs = ['TSS', 'ARCA', 'COCO', 'PSYNSW' ]
# atom_df = atom_df[~atom_df['Program'].isin(non_nsw_programs)] 


In [19]:
len(ep_df), len(atom_df) 

(797, 3147)

In [20]:
atom_df.Program.value_counts()

Program
EUROPATH    621
GOLBICE     527
MURMICE     459
SAPPHIRE    408
GOLBGNRL    367
BEGAPATH    223
MURMPP      198
MURMWIO     197
MONPATH     118
MURMHEAD     29
Name: count, dtype: int64

## NADA matching

In [63]:
# from match_audit import match_assessments
from data_prep import get_stage_per_episode


In [ ]:
matching_ndays_slack = 7

In [30]:
# def match_assessments(episodes_df, atoms_df): #pdc_substance_mapping
#     # Apply the mapping to the ESTABLISHMENT IDENTIFIER and PDCSubstanceOfConcern columns in episodes_df
#     # episodes_df['ESTABLISHMENT IDENTIFIER'] = episodes_df['ESTABLISHMENT IDENTIFIER'].map(establishment_program_mapping)
#     # episodes_df['PDCSubstanceOfConcern'] = episodes_df['PDCSubstanceOfConcern'].map(pdc_substance_mapping)

#     # Merge the dataframes on SLK and Program
#     merged_df = pd.merge(episodes_df, atoms_df, how='inner', left_on=['SLK', 'Program'], right_on=['SLK', 'Program'])

#     # Filter rows where AssessmentDate falls within CommencementDate and EndDate (or after CommencementDate if EndDate is NaN)
#     matched_df = merged_df.loc[((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
#                                 (merged_df['AssessmentDate'] <= merged_df['EndDate'])) |
#                                ((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) & 
#                                 (merged_df['EndDate'].isna()))]

#     # Check if PDCSubstanceOfConcern matches
#     # mismatched_df = matched_df.loc[matched_df['PDCSubstanceOfConcern_x'] != matched_df['PDCSubstanceOfConcern_y']]

#     # if len(mismatched_df) > 0:
#     #     logger.info(f"There are {len(mismatched_df)} rows where PDCSubstanceOfConcern does not match.")
#     #     logger.info(mismatched_df)

#     return matched_df

In [65]:
def get_mask_datefit(row, slack_days:int = 0):
    # Convert to datetime if not already in that format
    # assessment_date = pd.to_datetime(row['AssessmentDate'], errors='coerce')
    # commencement_date = pd.to_datetime(row['CommencementDate'], errors='coerce')
    # end_date = pd.to_datetime(row['EndDate'], errors='coerce')

    # Create a Timedelta for slack days
    slack_td = pd.Timedelta(days=slack_days)

    # Check conditions
    after_commencement = row['AssessmentDate'].date() >= (row['CommencementDate'] - slack_td)
    before_end_date = row['AssessmentDate'].date() <= (row['EndDate'] + slack_td)

    return after_commencement and before_end_date


def match_assessments(episodes_df, atoms_df, matching_ndays_slack: int=0):

    # Merge the dataframes on SLK and Program
    df = pd.merge(episodes_df, atoms_df, how='inner', left_on=[
                  'SLK', 'Program'], right_on=['SLK', 'Program'])

    # Filter rows where AssessmentDate falls within CommencementDate and EndDate (or after CommencementDate if EndDate is NaN)
    mask = df.apply(get_mask_datefit, slack_days=matching_ndays_slack, axis=1)
    # mask = get_mask_datefit(df['AssessmentDate'], df['CommencementDate'],
    #                         df['EndDate'], slack_days=matching_ndays_slack)
    filtered_df = df[mask]
    # matched_df = merged_df.loc[((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) &
    #                             (merged_df['AssessmentDate'] <= merged_df['EndDate'])) |
    #                            ((merged_df['AssessmentDate'] >= merged_df['CommencementDate']) &
    #                             (merged_df['EndDate'].isna()))]

    # Check if PDCSubstanceOfConcern matches
    # mismatched_df = matched_df.loc[matched_df['PDCSubstanceOfConcern_x'] != matched_df['PDCSubstanceOfConcern_y']]

    # if len(mismatched_df) > 0:
    #     logger.info(f"There are {len(mismatched_df)} rows where PDCSubstanceOfConcern does not match.")
    #     logger.info(mismatched_df)

    return filtered_df

## Prevent matching a single ATOM to multiple episodes
   Do the following with increasing slack :  
   1.  Set the Slack to zero 
       i.e.Do a perfect match - ATOM assessment dates HAVE to fall within Episode dates (slack = 0) 


   2. In a loop with increasing slack 
      Match the remaining ATOMs with episodes.

In [66]:
matching_ndays_slack =0
matched_df = match_assessments(ep_df, atom_df, matching_ndays_slack) # no slack

In [67]:
matched_slks  = matched_df.SLK.unique()


In [85]:
matching_ndays_slack = 0 
unmatched_atoms = atom_df
result_matched_dfs = []


while matching_ndays_slack <= 7:
    # Get matched assessments with the current slack
    matched_df = match_assessments(ep_df, unmatched_atoms, matching_ndays_slack)

    # Add the matched DataFrame to the list
    result_matched_dfs.append(matched_df)

    # Update unmatched_atoms by filtering out matched SLKs from the current unmatched_atoms
    matched_slks = matched_df.SLK.unique()
    unmatched_atoms = unmatched_atoms[~unmatched_atoms.SLK.isin(matched_slks)]
    # there may be other assessments for this SLK that can match if the slack dways are increased
    # don't exclude the SLK, but the SLK +RowKey

    # Increment the slack days for the next iteration
    matching_ndays_slack += 1

# Concatenate all matched DataFrames from the list
result_matched_df = pd.concat(result_matched_dfs, ignore_index=True)

# while matching_ndays_slack <=7:
#   matched_df = match_assessments(ep_df, unmatched_atoms, matching_ndays_slack) # no slack
#   # remaining atoms
#   matched_slks  = matched_df.SLK.unique()
#   unmatched_atoms = atom_df[ ~atom_df.SLK.isin(matched_df.SLK.unique())]

#   result_matched_df = result_matched_df.append(matched_df)
#   matching_ndays_slack = matching_ndays_slack + 1


In [88]:
unmatched_atoms.RowKey

3406     GOLBICE_ITSP_20200702
2671    BEGAPATH_ITSP_15072020
2850     MONPATH_INAS_17072020
1853    EUROPATH_ITSP_30072020
3324    EUROPATH_ITSP_30072020
                 ...          
2       EUROPATH_INAS_20231028
2750      MURMPP_INAS_20231109
239      GOLBICE_INAS_20231112
4783     MURMWIO_INAS_20231113
1742    SAPPHIRE_ITSP_20231204
Name: RowKey, Length: 1359, dtype: object

In [86]:
len(result_matched_df)

1140

In [60]:
len(atom_df)

3147

In [87]:
len(unmatched_atoms)

1359

In [69]:
# unmatched having an SLK in ep_df:
unmatched_atoms_withep = unmatched_atoms[ unmatched_atoms.SLK.isin(ep_df.SLK.unique())]

In [79]:
unmatched_pc = round(len(unmatched_atoms)*100/len(atom_df),1)

In [82]:
# episodes without ATOM match
eps_no_assessment = ep_df[ ~ep_df.SLK.isin(matched_slks)]

In [83]:
eps_no_assessment

,AgencyCode,GEOGRAPHICAL LOCATION,PMSEpisodeID,PMSPersonID,PDCCode,PDCSubstanceOfConcern,CommencementDate,EndDate,SLK,Program
4,12QQ03076,10550,53719,139,2101,Alcohol,2022-09-15,2023-09-14,OL2IR141219712,SAPPHIRE
22,12QQ03076,10550,73361,9048,2101,Alcohol,2023-05-03,2024-01-19,UT2ON220719881,SAPPHIRE
52,12QQ03076,10550,92134,9708,3103,Methamphetamine,2023-10-25,2024-01-19,ALEAT141119982,SAPPHIRE
54,12QQ03076,10550,82142,9681,2101,Alcohol,2023-08-03,2024-01-19,AGIAN010119782,SAPPHIRE
59,12QQ03076,10550,82497,5087,2101,Alcohol,2023-07-27,2024-01-19,HAKOS301219941,SAPPHIRE
...,...,...,...,...,...,...,...,...,...,...
784,12KK03024,17751,87331,8994,3103,Methamphetamine,2023-09-18,2023-10-06,EWAIC130619862,MURMPP
786,12KK03024,17751,91756,9927,2101,Alcohol,2023-10-24,2024-01-19,TETAN050819711,MURMPP
788,12KK03024,17751,93874,10000,2499,"Benzodiazepines, nec",2023-11-08,2023-12-04,MIHAY100420012,MURMPP
791,12KK03024,17751,97488,10060,7101,Cannabinoids,2023-12-07,2024-01-19,EWTDA190220041,MURMPP


In [80]:

print(f"Unmatched Atoms({len(unmatched_atoms)} - {unmatched_pc}% )  without Episode (slack {matching_ndays_slack}) = {len(unmatched_atoms_withep)},\
       # SLKs unmatched : {len(unmatched_atoms_withep['SLK'].unique())}")

Unmatched Atoms(1381 - 43.9% )  without Episode (slack 0) = 94,       # SLKs unmatched : 42


42

In [21]:
len(matched_df.SLK.unique()), len(atom_df.SLK.unique())

(597, 1225)

In [23]:

matched_df = match_assessments(ep_df, atom_df, matching_ndays_slack)

In [24]:
# matched_df[['SLK', 'CommencementDate', 'AssessmentDate', 'EndDate']].head(30)
matched_df

,AgencyCode,GEOGRAPHICAL LOCATION,PMSEpisodeID,PMSPersonID,PDCCode,PDCSubstanceOfConcern,CommencementDate,EndDate,SLK,Program,...,Another Drug2,Another Drug2_DaysInLast28,Another Drug2_TypicalQtyStr,ATOPRiskEviction,ATOPHomeless,Past4Wk_ViolentToYou,PrimaryCaregiver_0-5,PrimaryCaregiver_5-15,PaidWorkDays,StudyDays
0,12QQ03076,10550,23373,143,2101,Alcohol,2021-12-16,2023-09-14,AMOAT241019791,SAPPHIRE,...,NaN,NaN,NaN,None,None,None,0,1,None,None
1,12QQ03076,10550,23373,143,2101,Alcohol,2021-12-16,2023-09-14,AMOAT241019791,SAPPHIRE,...,NaN,NaN,NaN,None,None,None,0,1,2,None
3,12QQ03076,10550,48350,5324,2101,Alcohol,2022-08-03,2023-08-02,UCELE131119691,SAPPHIRE,...,NaN,NaN,NaN,0,0,0,None,None,None,None
4,12QQ03076,10550,48350,5324,2101,Alcohol,2022-08-03,2023-08-02,UCELE131119691,SAPPHIRE,...,NaN,NaN,NaN,0,0,1,None,None,None,None
5,12QQ03076,10550,48350,5324,2101,Alcohol,2022-08-03,2023-08-02,UCELE131119691,SAPPHIRE,...,NaN,NaN,NaN,0,0,0,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,12KK03024,17751,101485,2093,2101,Alcohol,2022-11-18,2023-11-17,ARIYL210819911,MURMPP,...,NaN,NaN,NaN,None,None,None,None,None,18,None
1872,12KK03024,17751,101485,2093,2101,Alcohol,2022-11-18,2023-11-17,ARIYL210819911,MURMPP,...,NaN,NaN,NaN,0,0,0,None,None,0,None
1873,12KK03024,17751,101485,2093,2101,Alcohol,2022-11-18,2023-11-17,ARIYL210819911,MURMPP,...,NaN,NaN,NaN,0,0,0,None,None,28,None
1876,12KK03024,17751,101535,3899,3103,Methamphetamine,2023-04-06,2023-11-16,YGAUK280819881,MURMPP,...,NaN,NaN,NaN,None,None,None,None,None,6,None


In [39]:
# PDCSubstanceOrGambling atom_df.columns
# ep_df.columns # PDCSubstanceOfConcern # PDCCode <- from episode

# matched_df.PDC <- from ATOM
# matched_df.PDCSubstanceOfConcern <- from Episode

0              Alcohol
1              Alcohol
2              Alcohol
3              Alcohol
4              Alcohol
            ...       
738            Alcohol
739    Methamphetamine
740    Methamphetamine
741       Cannabinoids
742    Methamphetamine
Name: PDCSubstanceOfConcern, Length: 690, dtype: object

In [25]:
# list(matched_df.PDC.unique())
# list(ep_df.PDCSubstanceOfConcern.unique())
ep_df.PDCSubstanceOfConcern.value_counts()

PDCSubstanceOfConcern
Alcohol                                340
Methamphetamine                        274
Cannabinoids                           102
Cannabinoids and Related Drugs, nec     16
Nicotine/tobacco                         6
Heroin                                   6
Amphetamine                              5
Benzodiazepines, nec                     5
Cocaine                                  3
MDMA/Ecstasy                             3
Pharmaceutical opioids, nfd              2
Other Volatile Solvents, nec             2
Gambling                                 2
Opioid Antagonists, nec                  1
Acetone                                  1
Dexamphetamine                           1
Cannabinoid agonists                     1
Buprenorphine                            1
Alprazolam                               1
Meperidine analogues                     1
Name: count, dtype: int64

In [46]:
# import numpy as np

# conditions = [
#     # matched_df['PDCSubstanceOfConcern'].isin(['Ethanol', 'Alcohols, n.e.c.']),
#      matched_df['PDCSubstanceOfConcern'].isin(['Alcohol']),
#     matched_df['PDCCode'].between(7100, 7199),
#     matched_df['PDCCode'].between(3100, 3199),
#     matched_df['PDCSubstanceOfConcern'].isin(['Pharmaceutical Opioids, n.f.d.']),
#     matched_df['PDCCode'].between(2400, 2499),
#     matched_df['PDCCode'] == 1202,
#     matched_df['PDCCode'].between(1100, 1399),
#     ~matched_df['PDCSubstanceOfConcern'].isin(['Nicotine', 'Cocaine'])
# ]
# # 'Alcohol', 'Cannabinoids and Related Drugs, nec', 'Heroin',
# #        'Cannabinoids', 'Benzodiazepines, nec', 'Methamphetamine',
# #        'Nicotine/tobacco', 'Cocaine', nan, 'Dexamphetamine',
# #        'Amphetamine', 'Other Volatile Solvents, nec', 'MDMA/Ecstasy',
# #        'Gambling', 'Alprazolam', 'Cannabinoid agonists', 'Buprenorphine'

# choices = [
#     'Alcohol',
#     'Cannabis',
#     'Amphetamines',
#     'Other Opioid Use',
#     'Tranquilisers',
#     'Heroin',
#     'Other Opioid Use',
#     'Another drug use'
# ]

# matched_df['PDCFinal'] = np.select(conditions, choices, default=matched_df['PDCSubstanceOfConcern'])

In [26]:
# Since the primary key in episodes_df is PMSEpisodeID, and in atoms_df it is a combination of SLK and Program, 
# you can use the groupby method on the merged_df to group by SLK and Program, 
# and then check if there is more than one unique PMSEpisodeID for any group.

# grouped = matched_df.groupby(['SLK', 'Program'])['PMSEpisodeID'].nunique()
grouped = matched_df.groupby(['SLK', 'RowKey'])['PMSEpisodeID'].nunique()
duplicates = grouped[grouped > 1]


In [27]:
duplicates

SLK             RowKey                
AHYAT141219641  SAPPHIRE_ITSP_20231128    2
AYULA290319921  SAPPHIRE_ITSP_20231005    2
AZNUK310119901  GOLBICE_ITSP_20230912     2
EGEUS110319582  BEGAPATH_ITSP_20230419    2
Name: PMSEpisodeID, dtype: int64

In [47]:
# SLK + Program
onedupe = matched_df.loc[(matched_df.SLK =='AHYAT141219641') & (matched_df.Program == 'SAPPHIRE')]

In [48]:
onedupe [['Program', 'SLK', 'CommencementDate', 'AssessmentDate', 'EndDate', 'PMSEpisodeID', 'RowKey']]

,Program,SLK,CommencementDate,AssessmentDate,EndDate,PMSEpisodeID,RowKey
192,SAPPHIRE,AHYAT141219641,2023-07-11,11072023,2023-11-27,79760,SAPPHIRE_INAS_20230711
193,SAPPHIRE,AHYAT141219641,2023-07-11,19092023,2023-11-27,79760,SAPPHIRE_ITSP_20230919
194,SAPPHIRE,AHYAT141219641,2023-07-11,28112023,2023-11-27,79760,SAPPHIRE_ITSP_20231128
197,SAPPHIRE,AHYAT141219641,2023-11-28,28112023,2024-01-18,99200,SAPPHIRE_ITSP_20231128


In [34]:

# Get the keys for the duplicate rows
duplicate_keys = duplicates.index

# Filter matched_df to show only rows that match the duplicate keys
duplicate_rows_df = matched_df[matched_df.set_index(['SLK', 'RowKey']).index.isin(duplicate_keys)]

# Show all rows in the resulting DataFrame
# pd.set_option('display.max_rows', None)  # Set this to display all rows



In [35]:
duplicate_rows_df[['SLK','RowKey',  'Program'  ,  'CommencementDate',   'EndDate' , 'PMSEpisodeID', 'AssessmentDate' ]]

,SLK,RowKey,Program,CommencementDate,EndDate,PMSEpisodeID,AssessmentDate
65,AHYAT141219641,SAPPHIRE_ITSP_20231128,SAPPHIRE,2023-07-11,2023-11-27,79760,2023-11-28
68,AHYAT141219641,SAPPHIRE_ITSP_20231128,SAPPHIRE,2023-11-28,2024-01-08,99200,2023-11-28


In [28]:
matched_df['Program'].value_counts()

Program
EUROPATH    237
SAPPHIRE    218
MURMICE     194
GOLBICE     170
GOLBGNRL    115
MURMWIO      75
MURMPP       66
MONPATH      58
BEGAPATH     28
MURMHEAD      1
Name: count, dtype: int64

In [43]:
# matched_df.to_csv(f"{results_folder}matched_df.csv", index=False)

In [29]:
matched_df.PMSEpisodeID.unique().shape

(642,)

In [ ]:
# atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)].count()

In [32]:
# len(atom_df[ (atom_df.AssessmentDate >= active_clients_start_date ) & (atom_df.AssessmentDate <= active_clients_end_date)])

818

In [23]:
atom_df[atom_df['SLK'] =='OLOAC010820061'][[c for c in atom_df.columns if 'Nicotine_' in c]]

,Nicotine_DaysInLast28,Nicotine_PerOccassionUse,Nicotine_TypicalQtyStr
3116,28,None,
3117,28,None,


In [ ]:
# len(matched_df.PMSEpisodeID.unique())

In [30]:
matched_df.to_csv('./data/out/NADA_Matched_20220701-20231231.csv')

In [40]:
# df = pd.read_csv('./data/out/NADA_Matched_20230701-2023-12-31.csv')

In [31]:
# matched_df['Past4WkEngagedInOtheractivities.Paid Work'][0:2].apply(lambda x : x['Days'] if 'Days' in x)
matched_df.columns

Index(['AgencyCode', 'GEOGRAPHICAL LOCATION', 'PMSEpisodeID', 'PMSPersonID',
       'PDCCode', 'PDCSubstanceOfConcern', 'CommencementDate', 'EndDate',
       'SLK', 'Program', 'RowKey', 'Staff', 'AssessmentDate',
       'SDSIsAODUseOutOfControl', 'SDSHowMuchDoYouWorryAboutAODUse',
       'SDSDoesMissingFixMakeAnxious', 'SDSDoYouWishToStop',
       'SDSHowDifficultToStopOrGoWithout', 'K10Q01', 'K10Q02', 'K10Q03',
       'K10Q04', 'K10Q05', 'K10Q06', 'K10Q07', 'K10Q08', 'K10Q09', 'K10Q10',
       'AssessmentType', 'Past4WkPhysicalHealth', 'Past4WkMentalHealth',
       'Past4WkHaveYouViolenceAbusive', 'Past4WkBeenArrested',
       'Past4WkQualityOfLifeScore', 'Past4WkNumInjectingDays',
       'Alcohol_DaysInLast28', 'Alcohol_PerOccassionUse',
       'Alcohol_TypicalQtyStr', 'Amphetamines_DaysInLast28',
       'Amphetamines_TypicalQtyStr', 'Another Drug1',
       'Another Drug1_DaysInLast28', 'Another Drug1_TypicalQtyStr',
       'Cannabis_DaysInLast28', 'Cannabis_TypicalQtyStr',
       'N

In [32]:
from data_config import nada_final_fields

In [33]:
matched_df['AgencyCode'].value_counts()

AgencyCode
12QQ03062    237
12QQ03076    218
13K034       194
13Q035       170
12QQ03022    115
12KK03025     75
12KK03024     66
12QQ03061     58
12QQ03063     28
12KK03023      1
Name: count, dtype: int64

In [34]:
matched_df["Stage"] = get_stage_per_episode(matched_df)

In [35]:
min(matched_df['AssessmentDate']), max(matched_df['AssessmentDate'])

(Timestamp('2020-07-10 00:00:00'), Timestamp('2023-12-22 00:00:00'))

In [36]:
matched_df['AssessmentDate'] = matched_df['AssessmentDate'].dt.strftime('%d%m%Y')


In [29]:
matched_df['ATOPHomeless']

0         0
1         0
2         0
3         0
4         0
       ... 
899    None
900       0
901    None
902       0
903    None
Name: ATOPHomeless, Length: 663, dtype: object

In [37]:
df_final = pd.DataFrame(columns=nada_final_fields)

for column in nada_final_fields:
    if column in matched_df.columns:
        df_final[column] = matched_df[column]  # Or use another default value
    else:
        df_final[column] =""


In [ ]:
# from utils.df_ops_base import drop_fields
# debug_cols_to_drop =['GEOGRAPHICAL LOCATION', 'PDCSubstanceOfConcern','CommencementDate' ,'EndDate','SLK','Program','RowKey', 'Staff','AssessmentType']
# nada_final_df = drop_fields(df_final,debug_cols_to_drop)

In [35]:
for mycol in df_final.columns:
  if not mycol in nada_final_fields:
    print(mycol)

In [ ]:
# ddmmyyyy - assesment date
# stage

In [36]:
df_final.columns

Index(['AgencyCode', 'PMSEpisodeID', 'PMSPersonID', 'Stage', 'AssessmentDate',
       'PDCCode', 'SDSIsAODUseOutOfControl', 'SDSDoesMissingFixMakeAnxious',
       'SDSHowMuchDoYouWorryAboutAODUse', 'SDSDoYouWishToStop',
       ...
       'ATOPHomeless', 'ATOPRiskEviction', 'PrimaryCaregiver_0-5',
       'PrimaryCaregiver_5-15', 'Past4WkBeenArrested', 'Past4Wk_ViolentToYou',
       'Past4WkHaveYouViolenceAbusive', 'Past4WkMentalHealth',
       'Past4WkPhysicalHealth', 'Past4WkQualityOfLifeScore'],
      dtype='object', length=160)

array(['12QQ03076', '12QQ03063', '12QQ03061', '12QQ03062', '12QQ03022',
       '13Q035', '13K034', '12KK03025', '12KK03024'], dtype=object)

In [38]:
survy_text_fname = f"{active_clients_start_date}_{active_clients_end_date}"

In [39]:
df_final['PMSPersonID'] = df_final['PMSPersonID'].astype(str).apply(lambda x: x.zfill(4))


In [40]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1162 entries, 0 to 1877
Columns: 160 entries, AgencyCode to Past4WkQualityOfLifeScore
dtypes: float64(8), int64(3), object(149)
memory usage: 1.4+ MB


In [41]:
float_cols = df_final.select_dtypes(include=['float']).columns
df_final[float_cols] = df_final[float_cols].astype('Int64')
# float_cols = df.select_dtypes(include=['float']).columns
# df[float_cols] = df[float_cols].astype('Int64')

In [42]:
out_path = f"./data/out/no-header/new_surveyTXT_{survy_text_fname}.csv"
df_final.to_csv(out_path,index=False,header=False)
out_path = f"./data/out/new_surveyTXT_{survy_text_fname}.csv"
df_final.to_csv(out_path,index=False)

In [35]:
my_agency_codes = df_final['AgencyCode'].unique()

for agency_code, program in EstablishmentID_Program.items():
  if not agency_code in my_agency_codes:
    continue
  out_path = f"./data/out/no-header/new_surveyTXT_{program}_{survy_text_fname}.csv"
  df_final[df_final.AgencyCode == agency_code].to_csv(out_path,index=False,header=False)
  out_path = f"./data/out/new_surveyTXT_{program}_{survy_text_fname}.csv"
  df_final[df_final.AgencyCode == agency_code].to_csv(out_path,index=False)
  

In [32]:
fname

'20230701-20231231'

In [46]:
ep_df[ep_df.PMSEpisodeID == 79856]


,AgencyCode,GEOGRAPHICAL LOCATION,PMSEpisodeID,PMSPersonID,PDCCode,PDCSubstanceOfConcern,CommencementDate,EndDate,SLK,Program
44,12QQ03076,10550,79856,9527,2101,Alcohol,2023-07-11,2023-08-30,AW2AT050619771,SAPPHIRE


# stats

In [31]:
df_final['AgencyCode'].value_counts()

AgencyCode
12QQ03076    109
13K034        89
12QQ03062     78
13Q035        73
12QQ03022     42
12QQ03061     38
12KK03025     24
12KK03024     17
12QQ03063      8
Name: count, dtype: int64

In [ ]:

# my_agency_codes = df_final['AgencyCode'].unique()
# for agency_code, program in EstablishmentID_Program.items():
#   if not agency_code in my_agency_codes:
#     continue
#   print(df_final['AgencyCode'].)

In [38]:
program_mask = (atom_df['Program'] =='SAPPHIRE') 
mask =  (atom_df['Benzodiazepines_DaysInLast28'].notna()) & (atom_df['Benzodiazepines_DaysInLast28'] != "") 
fields = [c for c in atom_df.columns if 'Benzodiazepines' in c]
fields.append('Program')
atom_df[program_mask & mask][  fields]

,Benzodiazepines_TypicalQtyStr,Benzodiazepines_DaysInLast28,Program
1033,10.0; dosage (mgs),28,SAPPHIRE
638,17.0; dosage (mgs),28,SAPPHIRE
639,17.0; dosage (mgs),28,SAPPHIRE
640,17.0; dosage (mgs),28,SAPPHIRE
1255,1.0; pills,14,SAPPHIRE
1042,0.5; dosage (mgs),8,SAPPHIRE
1034,2.0; dosage (mgs),28,SAPPHIRE


In [32]:
# df_final.to_csv("./data/out/surveyTXT_20230701-2023-12-31.csv")